In [ ]:
import os 
from dotenv import load_dotenv
import requests
from google.oauth2 import service_account
import xmltodict
import json
import pandas as pd 
from requests import request
import pytz
from bs4 import BeautifulSoup
from typing import Dict
from google.oauth2 import service_account
from google.cloud import storage

import pyspark
from pyspark.sql import SparkSession
import os
import sys




In [ ]:
#load env variables
load_dotenv('../creds/.env', verbose=True, override=True)

In [ ]:


'''
----------------
INIT VARIABLES
----------------
'''

#setting up entsoe variables
security_token = os.environ.get("SECURITY_TOKEN")
ENTSOE_URL = 'https://transparency.entsoe.eu/api'

#setting up GCP variables
service_account_file = os.environ.get("SERVICE_ACCOUNT_FILE")
credentials = service_account.Credentials.from_service_account_file(
    service_account_file
)
gcs_bucket = os.environ.get("CLOUD_STORAGE_BUCKET")
print(gcs_bucket)

In [ ]:



#setting up session
entsoe_session = requests.Session()

'''
----------------
SETTING UP FUNCTION CALLS 
----------------
'''

#format dates
def datetime_to_str(dtm: pd.Timestamp) -> str:
    #convert timezone to UTC if it's exist and it's not on UTC already
    if dtm.tzinfo is not None and dtm.tzinfo != pytz.UTC:
        dtm = dtm.tz_convert("UTC")
    fmt = '%Y%m%d%H%M'
    dtm_str = dtm.strftime(fmt)
    return dtm_str

#basic requests
def basic_requests(params: Dict, start:pd.Timestamp, end: pd.Timestamp) -> requests.Response:
    #setting up time intervals start and stop
    start_str = datetime_to_str(start)
    end_str = datetime_to_str(end)

    #setting up params and extending with custom parameter based
    base_params = {
        'securityToken': security_token,
        'periodStart': start_str,
        'periodEnd': end_str,
    }
    params.update(base_params)
    
    #seting up sesssion
    session = requests.Session()
    response = session.get(url=ENTSOE_URL, params=params)

    return response

# upload data to GCS
def upload_blob_to_gcs(bucket_name, contents, destination_blob_name):
    # Upload file to bucket"""

    # ID of GCS bucket
    # bucket_name =

    # the contents from memory to be uploaded to file
    # contents =

    # the ID of your GCS object
    # destination_blob_name =

    storage_client = storage.Client(credentials=credentials)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_string(contents)


'''
----------------
EXTRACTION
----------------
'''
#for test, we'll be querying Actual Generation Output per Generation Unit

domain = '10YCZ-CEPS-----N'
params_requests = {
    'documentType': 'A73',
    'processType': 'A16',
    'in_Domain': {domain},
}
start=pd.Timestamp('202101011200', tz='Europe/Berlin')
end=pd.Timestamp('202101011300', tz='Europe/Berlin')

entsoe_data = basic_requests(params=params_requests, start=start, end=end)
# entsoe_dict = xmltodict.parse(entsoe_data.text)
# entsoe_json = json.dumps(entsoe_dict, indent=4)
# print(entsoe_json)
print(entsoe_data.text)

'''
----------------
LOAD
----------------
'''
# #upload to GCS
# landing_filename=f"entsoe_data_{start}.json"
# upload_blob_to_gcs(bucket_name=gcs_bucket, contents=entsoe_json, destination_blob_name=landing_filename)

#upload to GCS - XML
landing_filename=f"entsoe_data_{start}_{domain}.xml"
upload_blob_to_gcs(bucket_name=gcs_bucket, contents=entsoe_data.text, destination_blob_name=landing_filename)



In [6]:

#setup sparksession for entry point
spark = SparkSession.builder.master("local[4]").appName("gcp_playground").getOrCreate()

/usr/bin/env: ‘bash’: No such file or directory


RuntimeError: Java gateway process exited before sending its port number